In [1]:
import requests
from bs4 import BeautifulSoup 
import re
import time

place_name = ["小田原"]

/Users/abeyuuya/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [12]:
# DBファイルを保存するためのファイルパス

import sqlite3

# DBに接続する（指定したDBファイル存在しない場合は，新規に作成される）
con = sqlite3.connect("weather.sqlite")
cur = con.cursor()

# テーブルが存在しない場合は作成
cur.execute('''CREATE TABLE IF NOT EXISTS weather(
                date TEXT,
                precipitation FLOAT,
                temp_avg FLOAT, 
                temp_high FLOAT, 
                temp_low FLOAT, 
                humidity_avg FLOAT, 
                humidity_min FLOAT, 
                sunshine FLOAT)
                ''')

# DBへの接続を閉じる
con.commit()
con.close()

In [3]:
url = "https://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=46&block_no=1008&year=2023&month=12&day=&view=p1"

# 取ったデータをfloat型に変える(データが取れなかった場合、気象庁は"/"を埋め込んでいるため0に変える)
def str2float(str):
  try:
    return float(str)
  except:
    return 0.0

In [4]:
r = requests.get(url)
r.encoding = r.apparent_encoding

In [13]:
data_list = [['月日', '降水量', '気温_平均', '気温_最高', '気温_最低', '湿度_平均', '湿度_最小', '日照時間']]

month = 12
soup = BeautifulSoup(r.text, 'html.parser')
table = soup.find('table', {'id': 'tablefix1', 'class': 'data2_s'})
rows = table.findAll('tr', class_='mtx')
rows = rows[19:]

con = sqlite3.connect("weather.sqlite")
cur = con.cursor()


for row in rows:
    data = row.findAll('td')
    rowData = []
    rowData.append(str(month) + "/" + str(data[0].string))
    rowData.append(str(data[1].string) + "mm")
    rowData.append(str(data[4].string) + "°C")
    rowData.append(str(data[5].string) + "°C")
    rowData.append(str(data[6].string) + "°C")
    rowData.append(str(data[7].string) + "％")
    rowData.append(str(data[8].string) + "％")
    rowData.append(str(data[15].string) + "h")

    cur.execute("INSERT INTO weather (date, precipitation, temp_avg, temp_high, temp_low, humidity_avg, humidity_min, sunshine) VALUES (?, ?, ?, ?, ?, ?, ?, ?)", rowData)
    data_list.append(rowData)

if __name__ == "__main__":
    result = data_list
    print(result)

con.commit()
con.close()

[['月日', '降水量', '気温_平均', '気温_最高', '気温_最低', '湿度_平均', '湿度_最小', '日照時間'], ['12/17', '0.0mm', '10.6°C', '19.3°C', '3.7°C', '45％', '29％', '8.8h'], ['12/18', '0.0mm', '5.8°C', '11.2°C', '1.0°C', '61％', '23％', '6.7h'], ['12/19', '0.0mm', '6.6°C', '10.9°C', '2.1°C', '67％', '38％', '3.6h'], ['12/20', '0.0mm', '8.8°C', '14.0°C', '4.2°C', '73％', '46％', '6.8h'], ['12/21', '0.0mm', '7.7°C', '10.5°C', '2.8°C', '34％', '25％', '8.9h'], ['12/22', '0.0mm', '4.1°C', '10.1°C', '-1.4°C', '42％', '23％', '9.1h'], ['12/23', '0.0mm', '3.4°C', '10.6°C', '-1.7°C', '55％', '23％', '8.9h'], ['12/24', '0.0mm', '4.0°C', '9.4°C', '-0.6°C', '75％', '39％', '4.3h'], ['12/25', '0.0mm', '5.8°C', '14.3°C', '-0.6°C', '67％', '29％', '8.6h'], ['12/26', '0.0mm', '5.4°C', '13.1°C', '-1.2°C', '75％', '41％', '9.0h'], ['12/27', '0.0mm', '7.0°C', '13.0°C', '0.7°C', '72％', '44％', '5.3h'], ['12/28', '0.0mm', '6.6°C', '11.9°C', '2.8°C', '74％', '41％', '8.2h'], ['12/29', '0.0mm', '6.6°C', '14.0°C', '1.2°C', '78％', '39％', '9.0h'], ['12/30', '0.0mm

In [16]:
# DBに接続する
con = sqlite3.connect("weather.sqlite")
cur = con.cursor()

# SQLを用意
sql_select = "SELECT * FROM weather;"

# SQLを実行
cur.execute(sql_select)

for row in cur:
    print(row)

# DBへの接続を閉じる
con.close()

[['月日', '降水量', '気温_平均', '気温_最高', '気温_最低', '湿度_平均', '湿度_最小', '日照時間'], ['12/17', '0.0mm', '10.6°C', '19.3°C', '3.7°C', '45％', '29％', '8.8h'], ['12/18', '0.0mm', '5.8°C', '11.2°C', '1.0°C', '61％', '23％', '6.7h'], ['12/19', '0.0mm', '6.6°C', '10.9°C', '2.1°C', '67％', '38％', '3.6h'], ['12/20', '0.0mm', '8.8°C', '14.0°C', '4.2°C', '73％', '46％', '6.8h'], ['12/21', '0.0mm', '7.7°C', '10.5°C', '2.8°C', '34％', '25％', '8.9h'], ['12/22', '0.0mm', '4.1°C', '10.1°C', '-1.4°C', '42％', '23％', '9.1h'], ['12/23', '0.0mm', '3.4°C', '10.6°C', '-1.7°C', '55％', '23％', '8.9h'], ['12/24', '0.0mm', '4.0°C', '9.4°C', '-0.6°C', '75％', '39％', '4.3h'], ['12/25', '0.0mm', '5.8°C', '14.3°C', '-0.6°C', '67％', '29％', '8.6h'], ['12/26', '0.0mm', '5.4°C', '13.1°C', '-1.2°C', '75％', '41％', '9.0h'], ['12/27', '0.0mm', '7.0°C', '13.0°C', '0.7°C', '72％', '44％', '5.3h'], ['12/28', '0.0mm', '6.6°C', '11.9°C', '2.8°C', '74％', '41％', '8.2h'], ['12/29', '0.0mm', '6.6°C', '14.0°C', '1.2°C', '78％', '39％', '9.0h'], ['12/30', '0.0mm